#  Random Forest algorithm
This tutorial builds on the decision tree tutorial to build the random forest algorithm. The tutorial follows from here: [link](https://machinelearningmastery.com/implement-random-forest-scratch-python/).

Dataset source: [link](https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/)

In [53]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np

In [25]:
# Load a csv file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [4]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [5]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    # assign integers for the classes
    for i, value in enumerate(unique):
        lookup[value] = i
    # replace the class names in the dataset with their corresponding integer codes
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [6]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [7]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [8]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        # construct test set
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [9]:
# Split a datset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [64]:
cost_function = 'entropy'
replace_attrib = True

In [11]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [26]:
# Cross entropy loss function for nodes
def entropy_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    e_split = 0.0
    epsilon = 1e-12
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        grp_entr = 0.0
        for clazz in classes:
            p = [row[-1] for row in group].count(clazz)/size
            if p == 0.0:
                p = epsilon
            grp_entr += -1.0 * p * np.log2(p)
        e_split += (size/n_instances) * grp_entr
    return e_split

In [62]:
# Select the best split point for a dataset
def get_split(dataset, n_features, used_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    # select the features or attributes to be used for splitting (without replacement)
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if replace_attrib:
            if index not in features:
                features.append(index)
        else:
            if index not in features and index not in used_features:
                features.append(index)
    # use the selected features to find the best splitting attribute and value
    for index in features:
        for row in dataset:
            groups  = test_split(index, row[index], dataset)
            # calculate cost
            if cost_function == 'gini':
                cost = gini_index(groups, class_values)
            else:
                cost = entropy_index(groups, class_values)
            if cost < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], cost, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups, 'score':b_score}

In [16]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [49]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth, used_features):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left+right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features, used_features)
        split(node['left'], max_depth, min_size, n_features, depth+1, used_features +[node['index']])
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features, used_features)
        split(node['right'], max_depth, min_size, n_features, depth+1, used_features +[node['index']])

In [44]:
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features, [])
    split(root, max_depth, min_size, n_features, 1, [root['index']])
    return root

In [37]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [29]:
# Create a random subsample from the dataset with replacement for Bagging
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [21]:
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    # voting
    return max(set(predictions), key=predictions.count)

In [23]:
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions

In [65]:
# Test the random forest algorithm
seed(2)
# load and prepare data
dataset = load_csv('sonar.all-data')
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10, 15, 20, 25]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [70.73170731707317, 53.65853658536586, 63.41463414634146, 68.29268292682927, 75.60975609756098]
Mean Accuracy: 66.341%
Trees: 5
Scores: [73.17073170731707, 75.60975609756098, 78.04878048780488, 80.48780487804879, 82.92682926829268]
Mean Accuracy: 78.049%
Trees: 10
Scores: [78.04878048780488, 70.73170731707317, 80.48780487804879, 75.60975609756098, 73.17073170731707]
Mean Accuracy: 75.610%
Trees: 15
Scores: [78.04878048780488, 82.92682926829268, 65.85365853658537, 68.29268292682927, 82.92682926829268]
Mean Accuracy: 75.610%
Trees: 20
Scores: [73.17073170731707, 78.04878048780488, 73.17073170731707, 80.48780487804879, 92.6829268292683]
Mean Accuracy: 79.512%
Trees: 25
Scores: [80.48780487804879, 80.48780487804879, 90.2439024390244, 75.60975609756098, 82.92682926829268]
Mean Accuracy: 81.951%


**with attribute replacement**

```Trees: 1
Scores: [56.09756097560976, 63.41463414634146, 60.97560975609756, 58.536585365853654, 73.17073170731707]
Mean Accuracy: 62.439%
Trees: 5
Scores: [70.73170731707317, 58.536585365853654, 85.36585365853658, 75.60975609756098, 63.41463414634146]
Mean Accuracy: 70.732%
Trees: 10
Scores: [82.92682926829268, 75.60975609756098, 97.5609756097561, 80.48780487804879, 68.29268292682927]
Mean Accuracy: 80.976%```

**without attribute replacement**

```Scores: [68.29268292682927, 63.41463414634146, 63.41463414634146, 60.97560975609756, 60.97560975609756]
Mean Accuracy: 63.415%
Trees: 5
Scores: [73.17073170731707, 73.17073170731707, 65.85365853658537, 78.04878048780488, 75.60975609756098]
Mean Accuracy: 73.171%
Trees: 10
Scores: [82.92682926829268, 80.48780487804879, 78.04878048780488, 73.17073170731707, 82.92682926829268]
Mean Accuracy: 79.512%```